## CPSC 483 (Section 02)

### Fall 2021 - Project 3

#### Group Members

- William Huynh

- Jake Sichley

- Ricky Segarra

##**Experiment 1**: reading pandas.read.csv() to load and examine the data set from bank-additional-full.csv and then using head to examine the first 5 rows. 


In [ ]:
import pandas as pd

data_set = pd.read_csv(r"bank-additional-full.csv", sep=";")
data_set.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


##**Experiment 2**: Using sklearn.model.selection.train_test_split() to split the features and target values into separate training and tests sets. 90% of the data is teh training set and 10% is the testing. To make sure it's reproducible, we pass the ekyword argument random_state = (2021-10-25). 

In [ ]:
from sklearn.model_selection import train_test_split

# extract our testing, training set
training_set, testing_set = train_test_split(data_set, train_size=0.9, random_state=(2021-10-25))

##**Experiment 3**: Preprocess trainig sets by removing duraiton column and setting aside feature y in traiing and testing, and then dropping it from features.




In [ ]:
# set aside our 'y'
training_set_y = training_set['y']
testing_set_y = testing_set['y']

# drop ['duration'], ['y'] from our DataFrames
training_set.drop(columns=['duration', 'y'], inplace=True)
testing_set.drop(columns=['duration', 'y'], inplace=True)

training_set.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
22975,56,retired,married,professional.course,no,yes,no,cellular,aug,mon,5,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1
14746,26,technician,single,professional.course,no,unknown,unknown,cellular,jul,wed,1,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1
12505,38,admin.,divorced,high.school,no,yes,no,cellular,jul,mon,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1
6801,33,admin.,divorced,university.degree,no,no,no,telephone,may,wed,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
18389,29,admin.,married,high.school,no,no,no,cellular,jul,thu,3,999,0,nonexistent,1.4,93.918,-42.7,4.968,5228.1


##**Experiment 4**: Making input variables described as "bank client data as features" using drop_first and pandas.get_dummies().


In [ ]:
bank_client_data_columns = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']

# extract the columns labeled as 'bank client data' from our training & testing sets
bank_client_data_training_set = training_set[bank_client_data_columns]
bank_client_data_testing_set = testing_set[bank_client_data_columns]

# encode our categorical variables
encoded_training_set = pd.get_dummies(bank_client_data_training_set, drop_first=True)
encoded_testing_set = pd.get_dummies(bank_client_data_testing_set, drop_first=True)

encoded_training_set.head()

,age,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes
22975,56,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
14746,26,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0
12505,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
6801,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
18389,29,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


##**Experiment 5**: Using scikit-learn to fit a Categorical Naive Bayes classifiers on features in 4. Afterwards score it on training and tests sets.

In [ ]:
from sklearn.naive_bayes import CategoricalNB

cnb = CategoricalNB()
cnb.fit(encoded_training_set, training_set_y)

print(cnb.score(encoded_training_set, training_set_y))
print(cnb.score(encoded_testing_set, testing_set_y))

0.8818689470986538
0.8788540907987376



The two scores were : 

*   Training | 0.8818689470986538
*   Testing  | 0.8788540907987376

These classifiers are highly accurate since they are 88%. 


##**Experiment 6**: Relooking at data in experiment 5 and finding how many categories there are. Then check if it's reasonable. 

In [ ]:
# pull the age column out of our encoded training set
age_series = encoded_training_set['age']

print(len(age_series.unique()))

78


There are 78 different categories. This is unreasonable because age has a wide range, and they can eb seperated into different categories. 

##**Experiment 7**: Splitting ages into bins, one per decade. Afterwards re-train classifier with the bins and check performance changes. 

In [ ]:
# define our decades
decades = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# verify we have 10 bins
age_series_as_decades = pd.cut(bank_client_data_training_set['age'], bins=decades)
print(age_series_as_decades.cat.categories.size)

# replace the original age data with our binned age date
bank_client_data_training_set_with_bins = bank_client_data_training_set.copy(deep=True)
bank_client_data_training_set_with_bins['age'] = age_series_as_decades

# repeat the process for our testing set
age_series_as_decades = pd.cut(bank_client_data_testing_set['age'], bins=decades)
bank_client_data_testing_set_with_bins = bank_client_data_testing_set.copy(deep=True)
bank_client_data_testing_set_with_bins['age'] = age_series_as_decades

10


In [ ]:
# re-encode our data
encoded_training_set_with_bins = pd.get_dummies(bank_client_data_training_set_with_bins, drop_first=True)
encoded_testing_set_with_bins = pd.get_dummies(bank_client_data_testing_set_with_bins, drop_first=True)

# retrain our classifier
cnb = CategoricalNB()
cnb.fit(encoded_training_set_with_bins, training_set_y)

print(cnb.score(encoded_training_set_with_bins, training_set_y))
print(cnb.score(encoded_testing_set_with_bins, testing_set_y))

0.881437319593191
0.8788540907987376


The new scores are 


*   Training | 0.881437319593191          
*   Testing  | 0.8788540907987376

Comparing this to the results of number 5 we can see there is a marginal difference. The performance, as the training set becomes worse and the testing set stays the same. 



##**Experiment 8**: Repeating experiment (8) with a KNN classifier and comparing results. 


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knb = KNeighborsClassifier()
knb.fit(encoded_training_set, training_set_y)

print(knb.score(encoded_training_set, training_set_y))
print(knb.score(encoded_testing_set, testing_set_y))

0.8903935903315439
0.8771546491866958


The scores from KNN are : 


*   Training | 0.8903935903315439
*   Testing  | 0.8771546491866958

Comparing it to the results from 5, the training set is slightly higher and and testing set is slgihtly lower. 


##**Experiment 9**: Checking how many values in the test set have repsonse 0 and how many are repsonse 1. Checking the score if we assumed no customer has ever subscribed to the product. 


In [ ]:
print(testing_set_y.value_counts())

no     3646
yes     473
Name: y, dtype: int64


The test set has 3646 0's and 473 1's. If we assumed that no customer ever subscribed to the rpoduct, the score would be high since the counts show that they most likely didn't subscribe. 

##**Experiment 10**: Using numpy.zeros_like() to create a target vector that reprsents output of "dumb classifier" of 9. Then we create a confusion matrix and find the AUC. 


In [ ]:
from numpy import array
from numpy import zeros_like

# create a fake response series
# training

modified_training_set_y = zeros_like(training_set_y)
modified_training_set_y[modified_training_set_y == 0] = 'no'

print(knb.score(encoded_training_set, modified_training_set_y))

# testing
modified_testing_set_y = zeros_like(testing_set_y)
modified_testing_set_y[modified_testing_set_y == 0] = 'no'

print(knb.score(encoded_testing_set, modified_testing_set_y))

0.9718363052685532
0.965768390386016


In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression

# -- SECTION: DUMB CLASSIFIER --
# ---- SUBSECTION: TRAINING -----

# create a confusion matrix with our "dumb" classifier
print(confusion_matrix(training_set_y, modified_training_set_y))

# score our "dumb" confusion matrix
modified_training_set_y[modified_training_set_y == 'no'] = 0

print(roc_auc_score(training_set_y, modified_training_set_y))

[[32902     0]
 [ 4167     0]]
0.5


The confusion matrix is :
  
[32902     0]

[ 4167     0]

The AUC is 0.5.

##**Experiment 11**: Creating confusion matricies and computing the AUC for each of the classifiers in expermients 7 and 8. Then check performance. 


In [ ]:
# -- SECTION: DUMB CLASSIFIER --
# ---- SUBSECTION: TESTING -----

# create a confusion matrix with our "dumb" classifier
print(confusion_matrix(testing_set_y, modified_testing_set_y))

# score our "dumb" confusion matrix
modified_testing_set_y[modified_testing_set_y == 'no'] = 0

print(roc_auc_score(testing_set_y, modified_testing_set_y))

[[3646    0]
 [ 473    0]]
0.5


Dumb Classifier


[3646    0]

[ 473    0]

0.5

In [ ]:
# -- SECTION: Categorical Naive Bayes CLASSIFIER --
# ---- SUBSECTION: TRAINING -----

# create a confusion matrix with our Categorical Naive Bayes classifier
cnb_training_predictions = cnb.predict(encoded_training_set_with_bins)

# create our cnb confusion matrix
print(confusion_matrix(training_set_y, cnb_training_predictions))

cnb_training_predictions[cnb_training_predictions == 'no'] = 0
cnb_training_predictions[cnb_training_predictions == 'yes'] = 1
cnb_training_predictions = cnb_training_predictions.astype(int)

# score our cnb confusion matrix
print(roc_auc_score(training_set_y, cnb_training_predictions))

[[32173   729]
 [ 3666   501]]
0.5490368368852783


In [ ]:
# -- SECTION: Categorical Naive Bayes CLASSIFIER --
# ---- SUBSECTION: TESTING -----

# create a confusion matrix with our Categorical Naive Bayes classifier
cnb_testing_predictions = cnb.predict(encoded_testing_set_with_bins)

# create our cnb confusion matrix
print(confusion_matrix(testing_set_y, cnb_testing_predictions))

cnb_testing_predictions[cnb_testing_predictions == 'no'] = 0
cnb_testing_predictions[cnb_testing_predictions == 'yes'] = 1
cnb_testing_predictions = cnb_testing_predictions.astype(int)

# score our cnb confusion matrix
print(roc_auc_score(testing_set_y, cnb_testing_predictions))

[[3554   92]
 [ 407   66]]
0.5571508757606297


Categorical Naive Bayes Classifier Confusion Matrix and AUC: 

TRAINING 

[32173   729]

[ 3666   501]

0.5490368368852783


TESTING 

[3554   92]

[ 407   66]

0.5571508757606297


In [ ]:
# -- SECTION: KNeighbors CLASSIFIER --
# ---- SUBSECTION: TRAINING -----

# create a confusion matrix with our KNeighbors Classifier
knb_training_predictions = knb.predict(encoded_training_set)

# create our knb confusion matrix
print(confusion_matrix(training_set_y, knb_training_predictions))

knb_training_predictions[knb_training_predictions == 'no'] = 0
knb_training_predictions[knb_training_predictions == 'yes'] = 1
knb_training_predictions = knb_training_predictions.astype(int)

# score our knb confusion matrix
print(roc_auc_score(training_set_y, knb_training_predictions))

[[32432   470]
 [ 3593   574]]
0.5617320670877847


In [ ]:
# -- SECTION: KNeighbors CLASSIFIER --
# ---- SUBSECTION: TESTING -----

# create a confusion matrix with our KNeighbors Classifier
knb_testing_predictions = knb.predict(encoded_testing_set)

# create our knb confusion matrix
print(confusion_matrix(testing_set_y, knb_testing_predictions))

knb_testing_predictions[knb_testing_predictions == 'no'] = 0
knb_testing_predictions[knb_testing_predictions == 'yes'] = 1
knb_testing_predictions = knb_testing_predictions.astype(int)

# score our knb confusion matrix
print(roc_auc_score(testing_set_y, knb_testing_predictions))

[[3559   87]
 [ 419   54]]
0.5451515692716626


KNN Classifier Confusion Matrix and AUC:

TRAINING

[32432   470]

[ 3593   574]

0.5617320670877847


TESTING

[3559   87]

[ 419   54]

0.5451515692716626


These classifiers seem to be doing alright. The majority of the data resulted in a correct prediciton, but 4167 were type 2 errors.


##**Experiment 12**: Working around imbalanced data with random oversampling, using pandas.DataFrame.where() and pandas.DataFrame.sample(). 


In [ ]:
# create a copy of our original data set
balanced_data_set = data_set.copy(deep=True)
# add a dummy column to allow for weight manipulation
balanced_data_set['weights'] = balanced_data_set['y']

# create pre-defined filters we can pass to pandas.DataFrame.where
filter_yes = balanced_data_set['y'] == 'yes'
filter_no = balanced_data_set['y'] == 'no'

# extract the count of 'yes' and 'no' responses
value_weights = balanced_data_set['y'].value_counts()
# dynamically calculate our weights based on distribution
adjusted_yes_weight = (value_weights[1]/(sum(value_weights)))
adjusted_no_weight = 1 - adjusted_yes_weight

# manipulate the values in our dummy weights column based on value
balanced_data_set['weights'] = balanced_data_set['weights'].where(filter_yes, other=adjusted_yes_weight)
balanced_data_set['weights'] = balanced_data_set['weights'].where(filter_no, other=adjusted_no_weight)

# create a new, "balanced" training set by sampling the data set using our generated weights
balanced_training_set = balanced_data_set.sample(n=len(training_set), weights=balanced_data_set['weights'], replace=True, random_state=(2021-10-25))

print(balanced_training_set.head())
print(balanced_training_set['y'].value_counts())

# finally, drop our dummy column
balanced_training_set.drop(columns=['weights'], inplace=True)

# set aside our 'y'
balanced_training_set_y = balanced_training_set['y']

# drop ['duration'], ['y'] from our DataFrames
balanced_training_set.drop(columns=['duration', 'y'], inplace=True)

       age         job  marital  ... nr.employed    y   weights
29898   38  technician   single  ...      5099.1   no  0.112654
4731    37  technician   single  ...      5191.0   no  0.112654
30861   34      admin.  married  ...      5099.1   no  0.112654
39531   30      admin.   single  ...      5008.7  yes  0.887346
28793   32      admin.   single  ...      5099.1   no  0.112654

[5 rows x 22 columns]
yes    18536
no     18533
Name: y, dtype: int64


##**Experiment 13**: Retraining both classifiers on balanced training sets, finding score, confusion matrix, and AUC for each, and then comparing performaence between classifiers. 


In [ ]:
# -- SECTION: DATA SET PRE-PROCESS FOR NAIVE BAYES --

# extract the columns labeled as 'bank client data' from our training & testing sets
bank_client_data_balanced_training_set = balanced_training_set[bank_client_data_columns]

balanced_age_series_as_decades = pd.cut(bank_client_data_balanced_training_set['age'], bins=decades)

bank_client_data_balanced_training_set_with_bins = bank_client_data_balanced_training_set.copy(deep=True)
bank_client_data_balanced_training_set_with_bins['age'] = balanced_age_series_as_decades

# encode our categorical variables
encoded_balanced_training_set_with_bins = pd.get_dummies(bank_client_data_balanced_training_set_with_bins, drop_first=True)

# -- SECTION: RETRAIN CNB WITH BALANCED AND ENCODED DATA --
balanced_cnb = CategoricalNB()
balanced_cnb.fit(encoded_balanced_training_set_with_bins, balanced_training_set_y)

print(balanced_cnb.score(encoded_balanced_training_set_with_bins, balanced_training_set_y))

0.6044673446815398


In [ ]:
# -- SECTION: RETRAINED CNB MATRIX, ROC_AUC --

# create a confusion matrix with our Categorical Naive Bayes classifier
cnb_balanced_training_predictions = balanced_cnb.predict(encoded_balanced_training_set_with_bins)

# create our cnb confusion matrix
print(confusion_matrix(balanced_training_set_y, cnb_balanced_training_predictions))

cnb_balanced_training_predictions[cnb_balanced_training_predictions == 'no'] = 0
cnb_balanced_training_predictions[cnb_balanced_training_predictions == 'yes'] = 1
cnb_balanced_training_predictions = cnb_balanced_training_predictions.astype(int)

# score our cnb confusion matrix
print(roc_auc_score(balanced_training_set_y, cnb_balanced_training_predictions))

[[11552  6981]
 [ 7681 10855]]
0.6044688703520166


In [ ]:
# -- SECTION: DATA SET PRE-PROCESS FOR KNeighbors CLASSIFIER  --

encoded_balanced_training_set = pd.get_dummies(bank_client_data_balanced_training_set, drop_first=True)

balanced_knb = KNeighborsClassifier()
balanced_knb.fit(encoded_balanced_training_set, balanced_training_set_y)

print(knb.score(encoded_balanced_training_set, balanced_training_set_y))

0.5582832015970217


In [ ]:
# -- SECTION: RETRAINED KNeighbors MATRIX, ROC_AUC --

# create a confusion matrix with our KNeighbors Classifier
knb_balanced_training_predictions = balanced_knb.predict(encoded_balanced_training_set)

# create our knb confusion matrix
print(confusion_matrix(balanced_training_set_y, knb_balanced_training_predictions))

knb_balanced_training_predictions[knb_balanced_training_predictions == 'no'] = 0
knb_balanced_training_predictions[knb_balanced_training_predictions == 'yes'] = 1
knb_balanced_training_predictions = knb_balanced_training_predictions.astype(int)

# score our knb confusion matrix
print(roc_auc_score(balanced_training_set_y, knb_balanced_training_predictions))

[[13216  5317]
 [ 2977 15559]]
0.7762499816317572


In [ ]:
# -- SECTION: Gaussian Naive Bayes --
from sklearn.naive_bayes import GaussianNB

social_and_economic_data_columns = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

social_and_economic_training_set = training_set[social_and_economic_data_columns]
social_and_economic_testing_set = testing_set[social_and_economic_data_columns]

gnb = GaussianNB()
gnb.fit(social_and_economic_training_set, training_set_y)

print(gnb.score(social_and_economic_training_set, training_set_y))
print(gnb.score(social_and_economic_testing_set, testing_set_y))

0.7199816558310178
0.7193493566399611


In [ ]:
# -- SECTION: Gaussian Naive Bayes MATRIX, ROC_AUC --

# create a confusion matrix with our KNeighbors Classifier
gnb_training_predictions = gnb.predict(social_and_economic_training_set)

# create our knb confusion matrix
print(confusion_matrix(training_set_y, gnb_training_predictions))

gnb_training_predictions[gnb_training_predictions == 'no'] = 0
gnb_training_predictions[gnb_training_predictions == 'yes'] = 1
gnb_training_predictions = gnb_training_predictions.astype(int)

# score our knb confusion matrix
print(roc_auc_score(training_set_y, gnb_training_predictions))

[[23717  9185]
 [ 1195  2972]]
0.7170302906069623


The K Neighbors classifier performed the best with an AUC score of 0.7762499816317572
.

##**Experiment 14**: Using Gaussian Naive Bayes to  check quantitive features of "social and economic context attributes". We then use the score, confusion matrix, and AUC, and then see how well data predicts the response. 


In [ ]:
# -- SECTION: Balanced Gaussian Naive Bayes --

balanced_social_and_economic_training_set = balanced_training_set[social_and_economic_data_columns]

balanced_gnb = GaussianNB()
balanced_gnb.fit(balanced_social_and_economic_training_set, balanced_training_set_y)

print(gnb.score(balanced_social_and_economic_training_set, balanced_training_set_y))

0.7190644473819094


The data predicts the reponse well with a score of 0.7190644473819094
.

##**Experiment 15**: Check if results of the last experiement changed if training set is balanced. 

In [ ]:
# -- SECTION: Balanced Gaussian Naive Bayes MATRIX, ROC_AUC --

# create a confusion matrix with our KNeighbors Classifier
gnb_balanced_training_predictions = balanced_gnb.predict(balanced_social_and_economic_training_set)

# create our knb confusion matrix
print(confusion_matrix(balanced_training_set_y, gnb_balanced_training_predictions))

gnb_balanced_training_predictions[gnb_balanced_training_predictions == 'no'] = 0
gnb_balanced_training_predictions[gnb_balanced_training_predictions == 'yes'] = 1
gnb_balanced_training_predictions = gnb_balanced_training_predictions.astype(int)

# score our knb confusion matrix
print(roc_auc_score(balanced_training_set_y, gnb_balanced_training_predictions))

[[13445  5088]
 [ 5326 13210]]
0.7190649651506402


The results of the last experiment changed to 0.7190649651506402. This change was negligible. 